In [ ]:
#execute just the first time to move scripts and dataset from input to output
import sys
from shutil import copytree, copyfile
sys.path.append("/kaggle/input/gta-dacs/")

src_datasets='/kaggle/input/gta-dacs/datasets'
dst_datasets='/kaggle/working/datasets/'
copytree(src_datasets, dst_datasets)
src_datasets='/kaggle/input/gta-dacs/models'
dst_datasets='/kaggle/working/models/'
copytree(src_datasets, dst_datasets)
#sys.path.append("/kaggle/input/cityscapes/")

# src_datasets='/kaggle/input/cityscapes/deeplab_resnet_pretrained_imagenet.pth'
# dst_datasets='/kaggle/working/deeplab_resnet_pretrained_imagenet.pth'
# copyfile(src_datasets, dst_datasets)

src_datasets='/kaggle/input/gta-dacs/transformmasks.py'
dst_datasets='/kaggle/working/transformmasks.py'
copyfile(src_datasets, dst_datasets)
src_datasets='/kaggle/input/gta-dacs/transformsgpu.py'
dst_datasets='/kaggle/working/transformgpu.py'
copyfile(src_datasets, dst_datasets)


copyfile(src = "/kaggle/input/gta-dacs/utils.py", dst = "/kaggle/working/utils.py")

'/kaggle/working/utils.py'

In [ ]:
src_datasets='/kaggle/input/gta-dacs/transformmasks.py'
dst_datasets='/kaggle/working/transformmasks.py'
copyfile(src_datasets, dst_datasets)
src_datasets='/kaggle/input/gta-dacs/transformsgpu.py'
dst_datasets='/kaggle/working/transformgpu.py'
copyfile(src_datasets, dst_datasets)

'/kaggle/working/transformgpu.py'

In [ ]:
from datasets.gta5 import GTA5
from datasets.cityscapes import CityScapes
from torch.utils.data import DataLoader
import torch.nn.functional as F
import transformmasks
import transformsgpu
import torch
import os
import numpy as np
import posixpath
from torchvision import transforms
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from models.bisenet.build_bisenet import BiSeNet
from utils import poly_lr_scheduler
IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)

def strong_transform(parameters, data=None, target=None):
    assert ((data is not None) or (target is not None))
    data, target = transformsgpu.oneMix(mask=parameters["Mix"], data=data, target=target)
    data, target = transformsgpu.colorJitter(colorJitter=parameters["ColorJitter"], img_mean=torch.from_numpy(IMG_MEAN.copy()).to(device), data=data, target=target)
    data, target = transformsgpu.gaussian_blur(blur=parameters["GaussianBlur"], data=data, target=target)
    data, target = transformsgpu.flip(flip=parameters["flip"], data=data, target=target)
    return data, target

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=19):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                #print("true class size",true_class.shape)
                #print("true label size",true_label.shape)
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

def convert_tensor_to_image(tensor):
    image = tensor.permute(1, 2, 0)
    return image

def train(model, optimizer, train_loader, target_loader, criterion):
    model.train()
    running_loss = 0.0
    iou_score = 0.0
    accuracy = 0.0
    ii = 0

    target_iter = iter(target_loader)
    max_target_iter=250
    target_counter=0
    for batch_idx, source_batch in enumerate(train_loader):
        if target_counter >= max_target_iter:
            target_iter = iter(target_loader)
            target_counter = 0
        ii += 1

        try:
            target_batch = next(target_iter)
        except StopIteration:
            target_iter = iter(target_loader)
            target_batch = next(target_iter)
        target_counter+=1
        inputs, targets = source_batch
        target_inputs, _ = target_batch

        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.float()
        targets = targets.squeeze()
        target_inputs = target_inputs.to(device).float()

        # Step 5: Generate pseudo-labels for target domain images
        with torch.no_grad():
            logits_u_w = model(target_inputs)[0]
            pseudo_label = torch.softmax(logits_u_w, dim=1)
            max_probs, targets_u_w = torch.max(pseudo_label, dim=1)

        # Step 6: Create mixed images and pseudo-labels
        batch_size = inputs.size(0)
        MixMask = []
        for image_i in range(batch_size):
            classes = torch.unique(targets[image_i])
            nclasses = classes.shape[0]
            selected_classes = classes[torch.Tensor(np.random.choice(nclasses, int((nclasses + nclasses % 2) / 2), replace=False)).long()].to(device)
            MixMask.append(transformmasks.generate_class_mask(targets[image_i], selected_classes).unsqueeze(0).to(device))

        inputs_u_s_list, targets_u_s_list = [], []
        for i, mask in enumerate(MixMask):
            strong_parameters = {"Mix": mask, "ColorJitter": 0.2, "GaussianBlur": 0.5, "flip": 0.5}
            inputs_u_s, targets_u_s = strong_transform(strong_parameters, data=torch.cat((inputs[i].unsqueeze(0), target_inputs[i].unsqueeze(0))), target=torch.cat((targets[i].unsqueeze(0), targets_u_w[i].unsqueeze(0))))
            inputs_u_s_list.append(inputs_u_s)
            targets_u_s_list.append(targets_u_s)

        inputs_u_s = torch.cat(inputs_u_s_list)
        targets_u_s = torch.cat(targets_u_s_list)

        # Step 7: Compute predictions for mixed images
        logits_u_s = model(inputs_u_s)[0]
        L_u = F.cross_entropy(logits_u_s, targets_u_s, ignore_index=255, reduction='none')
        mask = max_probs.ge(0.968).float()
        L_u = (L_u * mask).mean()

        # Compute predictions for source images
        outputs, _, _ = model(inputs)
        L_s = criterion(outputs.to(dtype=torch.float32), targets.to(dtype=torch.int64))

        # Combine supervised and unsupervised losses
        loss = L_s + L_u

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        iou_score += mIoU(outputs.to(device), targets.to(device))
        accuracy += pixel_accuracy(outputs.to(device), targets.to(device))

    train_loss = running_loss / len(train_loader)
    iou_score = iou_score / len(train_loader)
    accuracy = accuracy / len(train_loader)
    return train_loss, iou_score, accuracy
# Test loop
# calculate_label_prediction is a flag used to decide wether to calculate or not ground_truth and predicted tensor
def test(model, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    iou_score=0.0
    accuracy=0.0
    with torch.no_grad():
        for batch_idx,(inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.float()
            targets = targets.int()
            #Compute prediction and loss
            outputs = model(inputs)
            #print(batch_idx)
            loss = loss_fn(outputs.to(dtype=torch.float32), targets.squeeze().to(dtype=torch.int64))
            iou_score += mIoU(outputs.to(device), targets.to(device))
            accuracy += pixel_accuracy(outputs.to(device), targets.to(device))
            test_loss += loss.item()
            #break
    test_loss = test_loss / len(test_loader)
    iou_score = iou_score / len(test_loader)
    accuracy = accuracy / len(test_loader)
    #test_accuracy = 100. * correct / total
    return test_loss,iou_score,accuracy

print(CityScapes)
#dataset_path='/kaggle/input/cityscapes-polito/Cityscapes/Cityscapes/Cityspaces/'
gta_dataset_path='/kaggle/input/gta-dacs/GTA5/GTA5/GTA5/'
annotation_train=gta_dataset_path+'labels_correct/'
image_train=gta_dataset_path+'images/'
cityscapes_dataset_path='/kaggle/input/gta-dacs/Cityscapes/Cityscapes/Cityspaces/'
annotation_val=cityscapes_dataset_path+'gtFine/val'
image_val=cityscapes_dataset_path+'images/val'
annotation_val_train=cityscapes_dataset_path+'gtFine/train'
image_val_train=cityscapes_dataset_path+'images/train'
resize_transform_gta = v2.Resize(interpolation=transforms.InterpolationMode.NEAREST_EXACT,size = (720,1280))
resize_transform = v2.Resize(interpolation=transforms.InterpolationMode.NEAREST_EXACT,size = (512,1024))
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
applier_crop = v2.RandomApply(transforms=[v2.RandomCrop(size=(512, 1024))], p=0.5)
gta_train = GTA5(annotations_dir=annotation_train, images_dir=image_train,transform=resize_transform,applier=applier_crop)
cityscapes_val = CityScapes(annotations_dir=annotation_val, images_dir=image_val,transform=resize_transform,applier=applier_crop)
cityscapes_train=CityScapes(annotations_dir=annotation_val_train, images_dir=image_val_train,transform=resize_transform,applier=applier_crop)
train_loader = DataLoader(gta_train, batch_size=2, shuffle=False)
val_loader = DataLoader(cityscapes_val, batch_size=2, shuffle=False)
train_loader_cityscapes= DataLoader(cityscapes_train, batch_size=2, shuffle=False)
# Define the model and load it to the device
bisenet = BiSeNet(num_classes=19, context_path='resnet18')
bisenet.to(device)
optimizer = torch.optim.Adam(bisenet.parameters(), lr=0.001)
scheduler=poly_lr_scheduler(optimizer, 0.01, 1, lr_decay_iter=1, max_iter=50, power=0.9)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
epoch_beginning=0
epochs = 50

<class 'datasets.cityscapes.CityScapes'>
cuda


In [ ]:
train_loader_cityscapes.dataset.__len__()

1572

In [ ]:
# Define the model and load it to the device
model = BiSeNet(num_classes=19, context_path='resnet18')
model.to(device)
epoch = 0 #to initialize the lr
start_lr = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=start_lr)
l_rate = poly_lr_scheduler(optimizer, init_lr=start_lr , iter=epoch, lr_decay_iter=1, max_iter=50, power=0.9)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)

In [ ]:
#Freeze model parameters, but in avery fancy way
for index,param in enumerate(model.parameters()):
    if index < 75:
        param.requires_grad=False
#         print(index)
for param in model.attention_refinement_module2.parameters():
    param.requires_grad=False
for param in model.feature_fusion_module.parameters():
    param.requires_grad=True

In [ ]:
train_iou_list=[]
train_acc_list=[]
train_loss_list=[]
test_iou_list=[]
class_iou_list=[]
test_acc_list=[]
test_loss_list=[]

In [ ]:
src_datasets='/kaggle/input/weights-10-epoch/bisenet_GTA5_bs2_epoch10_weights.pth'
dst_datasets='/kaggle/working/bisenet_GTA5_bs2_epoch10_weights.pth'
copyfile(src_datasets, dst_datasets)

'/kaggle/working/bisenet_GTA5_bs2_epoch10_weights.pth'

In [ ]:
# Specifica il percorso del file dei pesi salvati
checkpoint_path = "bisenet_GTA5_bs2_epoch10_weights.pth"  # Aggiorna con il percorso corretto

# Carica i pesi nel modello
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [ ]:
# Train the model

bs=2
epoch_beginning = 10
epochs = 51
for epoch in range(epoch_beginning,epochs):
    n_epoch = epoch+1
    l_rate = poly_lr_scheduler(optimizer, init_lr=start_lr , iter=epoch, lr_decay_iter=1, max_iter=50, power=0.9)
    #train
    train_loss,train_iou,train_acc=train(model, optimizer, train_loader,train_loader_cityscapes, loss_fn)
    train_iou = np.nanmean(train_iou)
    train_iou_list.append(train_iou)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)

    #save model
    if n_epoch%5 == 0 or n_epoch==epochs:
        model_name = f"bisenet_GTA5_bs{bs}_epoch{n_epoch}_weights.pth"
        torch.save(model.state_dict(), model_name)

    #test
    test_loss,class_iou,test_acc = test(model, val_loader, loss_fn)
    test_iou = np.nanmean(class_iou)
    test_iou_list.append(test_iou)
    class_iou_list.append(class_iou)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)

    f = open("train_iou_list.txt", "a")
    f.write(str(train_iou)+ "\n")
    f.close()
    f = open("test_iou_list.txt", "a")
    f.write(str(test_iou)+ "\n")
    f.close()

    print(f"Epoch: {n_epoch}")
    print(f"- Train Acc: {train_acc:.3f}")
#     print(f"- Train Loss: {test_loss:.3f}")
    print(f"- Train mIoU: {train_iou:.3f}\n")
    print(f"- Test Acc: {test_acc:.3f}")
    print(f"- Test Loss: {test_loss:.3f}")
    print(f"- Test mIoU: {test_iou:.3f}")
    if n_epoch==1 or n_epoch == epochs:
        print(f"- Iou per Class: {class_iou}")
    print("__________________________")

Epoch: 11
- Train Acc: 0.489
- Train mIoU: 0.257

- Test Acc: 0.321
- Test Loss: 1.208
- Test mIoU: 0.184
__________________________
Epoch: 12
- Train Acc: 0.476
- Train mIoU: 0.252

- Test Acc: 0.322
- Test Loss: 1.286
- Test mIoU: 0.178
__________________________
Epoch: 13
- Train Acc: 0.470
- Train mIoU: 0.250

- Test Acc: 0.318
- Test Loss: 1.185
- Test mIoU: 0.186
__________________________
Epoch: 14
- Train Acc: 0.480
- Train mIoU: 0.254

- Test Acc: 0.332
- Test Loss: 1.203
- Test mIoU: 0.187
__________________________
Epoch: 15
- Train Acc: 0.476
- Train mIoU: 0.255

- Test Acc: 0.334
- Test Loss: 1.112
- Test mIoU: 0.188
__________________________
Epoch: 16
- Train Acc: 0.474
- Train mIoU: 0.254

- Test Acc: 0.327
- Test Loss: 1.224
- Test mIoU: 0.189
__________________________
Epoch: 17
- Train Acc: 0.467
- Train mIoU: 0.251

- Test Acc: 0.311
- Test Loss: 1.179
- Test mIoU: 0.182
__________________________
Epoch: 18
- Train Acc: 0.483
- Train mIoU: 0.256

- Test Acc: 0.311
-

Epoch: 1
- Train Acc: 0.435
- Train mIoU: 0.163

- Test Acc: 0.297
- Test Loss: 1.354
- Test mIoU: 0.138
- Iou per Class: 0.13785903419927545
__________________________
Epoch: 2
- Train Acc: 0.459
- Train mIoU: 0.210

- Test Acc: 0.326
- Test Loss: 1.303
- Test mIoU: 0.154
__________________________
Epoch: 3
- Train Acc: 0.471
- Train mIoU: 0.227

- Test Acc: 0.311
- Test Loss: 1.347
- Test mIoU: 0.160
__________________________
Epoch: 4
- Train Acc: 0.470
- Train mIoU: 0.234

- Test Acc: 0.298
- Test Loss: 1.414
- Test mIoU: 0.166
__________________________
Epoch: 5
- Train Acc: 0.472
- Train mIoU: 0.240

- Test Acc: 0.316
- Test Loss: 1.255
- Test mIoU: 0.172
__________________________
Epoch: 6
- Train Acc: 0.467
- Train mIoU: 0.242

- Test Acc: 0.279
- Test Loss: 1.842
- Test mIoU: 0.136
__________________________
Epoch: 7
- Train Acc: 0.462
- Train mIoU: 0.242

- Test Acc: 0.327
- Test Loss: 1.180
- Test mIoU: 0.174
__________________________
Epoch: 8
- Train Acc: 0.471
- Train mIoU: 0.248

- Test Acc: 0.302
- Test Loss: 1.414
- Test mIoU: 0.174
__________________________
Epoch: 9
- Train Acc: 0.467
- Train mIoU: 0.249

- Test Acc: 0.323
- Test Loss: 1.102
- Test mIoU: 0.186
__________________________
Epoch: 10
- Train Acc: 0.470
- Train mIoU: 0.255

- Test Acc: 0.311
- Test Loss: 1.249
- Test mIoU: 0.178
- Iou per Class: 0.17827379207672459